In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tqdm
!rm -rf sample_data
!rm -rf google_colab
!git clone https://github.com/fmaclean/google_colab.git
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
!conda update conda -y -q
!source /usr/local/etc/profile.d/conda.sh
!conda init 
!conda install -n root _license -y -q
!conda env create -f /content/google_colab/environment.yml
!source activate google_colab && conda install -c rdkit rdkit -y 
!cd google_colab && source activate google_colab && pip install -e .
!rm -rf google_colab/data
!rm -rf google_colab/models
!unzip drive/MyDrive/google_colab/data.zip -d google_colab/
!unzip drive/MyDrive/google_colab/models.zip -d google_colab/
%cd google_colab

Cloning into 'google_colab'...
remote: Enumerating objects: 22711, done.
remote: Counting objects: 100% (22711/22711), done.
remote: Compressing objects: 100% (15476/15476), done.
remote: Total 22711 (delta 6311), reused 22702 (delta 6302), pack-reused 0
Receiving objects: 100% (22711/22711), 45.93 MiB | 13.14 MiB/s, done.
Resolving deltas: 100% (6311/6311), done.
--2021-01-16 17:18:26--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-01-16 17:18:26--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303,

In [8]:
!rm -rf google_colab/data
!rm -rf google_colab/models
!unzip drive/MyDrive/google_colab/data.zip -d google_colab/
!unzip drive/MyDrive/google_colab/models.zip -d google_colab/

Archive:  drive/MyDrive/google_colab/data.zip
  inflating: google_colab/data/external/test.csv  
  inflating: google_colab/data/external/.gitkeep  
  inflating: google_colab/data/interim/.gitkeep  
  inflating: google_colab/data/processed/.gitkeep  
  inflating: google_colab/data/raw/.gitkeep  
Archive:  drive/MyDrive/google_colab/models.zip
  inflating: google_colab/models/.gitkeep  


In [15]:
!ls
!source activate google_colab && python src/colab.py

data		 LICENSE    README.md	      setup.py		   tox.ini
docs		 Makefile   references	      src		   upload.py
drive_auth.py	 models     reports	      src.egg-info
environment.yml  notebooks  requirements.txt  test_environment.py
/content/google_colab
   0  ...   2
0  4  ...  12

[1 rows x 3 columns]


In [16]:
import zipfile
import os
def _zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

def zipdir(folder):
  zipf = zipfile.ZipFile(f'{folder}.zip', 'w', zipfile.ZIP_DEFLATED)
  _zipdir(f'{folder}/', zipf)
  zipf.close()


[zipdir(folder) for folder in ['data','models']];
print("Files zipped")



Files zipped


In [17]:
%cd /content
!cd /content && cp -r google_colab/data.zip /content/drive/MyDrive/google_colab/
!cd /content && cp -r google_colab/models.zip /content/drive/MyDrive/google_colab/
print("Files copied to google drive")
drive.flush_and_unmount()

/content
Files copied to google drive
